# 加载环境

In [1]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# 首先导入所需第三方库
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import os
# 加载嵌入模型
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
# 建立索引
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate


# 配置LangSmith

In [2]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "Update to your API key"  # Update to your API key

from langsmith import Client

client = Client()

# 自定义大模型 Internlm-chat-7b-finetuned

In [3]:
class InternLM_LLM(LLM):
    # 基于本地 InternLM 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: InternLM 模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(torch.bfloat16).cuda()
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        system_prompt = """You are an AI assistant whose name is InternLM (书生·浦语).
        - InternLM (书生·浦语) is a conversational language model that is developed by Shanghai AI Laboratory (上海人工智能实验室). It is designed to be helpful, honest, and harmless.
        - InternLM (书生·浦语) can understand and communicate fluently in the language chosen by the user such as English and 中文.
        """
        
        messages = [(system_prompt, '')]
        response, history = self.model.chat(self.tokenizer, prompt , history=messages)
        return response
        
    @property
    def _llm_type(self) -> str:
        return "InternLM2"

# 加载大模型
#llm = InternLM_LLM("/root/share/model_repos/internlm2-chat-7b")
llm = InternLM_LLM("/root/project/bisai2/sft-guanfang-self/merged_7b_e10")

正在从本地加载模型...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

完成本地模型的加载


## 创建检索库(如果有)

In [4]:
# 加载已有数据库
embeddings = HuggingFaceEmbeddings(model_name="/root/model/jinaai/jina-embeddings-v2-base-zh")
vectordb = FAISS.load_local('./vectordb', embeddings,allow_dangerous_deserialization=True)

Some weights of BertModel were not initialized from the model checkpoint at /root/model/jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.

## 文本对话

In [21]:
# 建立一个链，该链接受问题和检索到的文档并生成答案
prompt = ChatPromptTemplate.from_template("""使用以上下文来回答用户的问题。总是使用中文回答。
<context>
{context}
</context>
如果给定的上下文无法让你做出回答，请对回答的内容以“*”开头，然后根据你的理解作答。
有用的回答: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


#对于给定的问题，我们可以使用检索器动态选择最相关的文档并将其传递进去

retriever = vectordb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# 返回结果
response = retrieval_chain.invoke({"input": "上汽集团有多少车辆搭载了德威系列的发动机"})
print(response["answer"])

截至今年上半年，上汽跃进有5800台左右的车辆搭载德威系列发动机。 
